In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score
from keras.layers import LSTM, Dense,Input,Layer
from keras import backend as K,Model
from numpy.random import seed
from imblearn.under_sampling import RandomUnderSampler 
from sklearn.preprocessing import StandardScaler
seed(0)
import os
os.environ['PYTHONHASHSEED']=str(0)
from sklearn.metrics import roc_curve,roc_auc_score
import tensorflow as tf
tf.random.set_seed(0)
import random
import random as python_random
python_random.seed(0)
random.seed(0)

In [2]:
data = pd.read_csv('creditcard.csv', na_filter=True)

data.drop(['Time'], axis=1, inplace=True)
selected_vars =['V4','V14','V10','V16','V9','V12','V11','V17','V7','V21','V18','V8','V1','V3','V6','V2','V27','V5','V26','Class']


In [3]:
tr_data = data[selected_vars]
tr_data.shape

(284807, 20)

In [4]:
X = tr_data.drop(['Class'], axis = 'columns')
Label_Data = tr_data['Class']

In [5]:
y=tr_data['Class']

In [20]:
from sklearn.linear_model import SGDOneClassSVM
from sklearn.linear_model import SGDOneClassSVM
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

for Test_size in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]:
    X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=Test_size, shuffle=False,random_state=0)
    SZ=StandardScaler()
    X_train=pd.DataFrame(SZ.fit_transform(X_train))
    X_test=pd.DataFrame(SZ.transform(X_test))

    df1_normal=X_train[y_train==0]
    clf = SGDOneClassSVM(nu=0.01808,random_state=0)
    res = clf.fit_predict(df1_normal)
    res[res==1] = 0
    res[res==-1] = 1
    ros = RandomUnderSampler(sampling_strategy=0.95)
    ros.fit_resample(df1_normal,res)
    index=ros.sample_indices_
    X_train.drop(X_train.index[index], inplace=True)
    y_train.drop(y_train.index[index], inplace=True)

    # X_train et X_test sont des dataframe qui contient les features
    train_LSTM_X=X_train.to_numpy()
    test_LSTM_X=X_test.to_numpy()
    train_LSTM_y=y_train.to_numpy()
    test_LSTM_y=y_test.to_numpy()
    w=3
    X_train=np.zeros([train_LSTM_X.shape[0]-w+1,w, train_LSTM_X.shape[1]])
    for i in range(train_LSTM_X.shape[0]-w+1):
        for j in range(w):
            X_train[i][j]=train_LSTM_X[i+j]
    X_test=np.zeros([test_LSTM_X.shape[0]-w+1,w, test_LSTM_X.shape[1]])
    for i in range(test_LSTM_X.shape[0]-w+1):
        for j in range(w):
            X_test[i][j]=test_LSTM_X[i+j]
    ## Recuperation des labels

    y_train=np.delete(train_LSTM_y,[*range(0, w-1, 1)], 0)
    y_test=np.delete(test_LSTM_y,[*range(0, w-1, 1)], 0)

    inputs=Input((w,19))
    x1=LSTM(50)(inputs)
    outputs=Dense(1,activation='sigmoid')(x1)
    model=Model(inputs,outputs)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['Precision','Recall'])
    history1=model.fit(X_train, y_train,epochs=5,batch_size=500,verbose=0)
    # predict probabilities for test set
    yhat_probs = model.predict(X_test, verbose=0)
    # reduce to 1d array
    yhat_probs =  yhat_probs[:, 0]> 0.5
    yhat_probs=yhat_probs.astype(int)

    print("q: " ,Test_size)
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_probs)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_probs)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_probs)
    print('Recall: %f' % recall)
    f1 = f1_score(y_test, yhat_probs)
    print('f1: %f' % f1)
    auc_score=roc_auc_score(y_test, yhat_probs)
    print('auc_score: %f' % auc_score)


q:  0.05
Accuracy: 0.999368
Precision: 0.600000
Recall: 0.545455
f1: 0.571429
auc_score: 0.772587
q:  0.1
Accuracy: 0.999684
Precision: 0.842105
Recall: 0.727273
f1: 0.780488
auc_score: 0.863584
q:  0.15
Accuracy: 0.999602
Precision: 0.926829
Recall: 0.730769
f1: 0.817204
auc_score: 0.865349
q:  0.2
Accuracy: 0.999579
Precision: 0.918033
Recall: 0.746667
f1: 0.823529
auc_score: 0.873289
q:  0.25
Accuracy: 0.999565
Precision: 0.956522
Recall: 0.702128
f1: 0.809816
auc_score: 0.851043
q:  0.3
Accuracy: 0.999602
Precision: 0.951220
Recall: 0.722222
f1: 0.821053
auc_score: 0.861088
q:  0.35
Accuracy: 0.999619
Precision: 0.947917
Recall: 0.733871
f1: 0.827273
auc_score: 0.866910
q:  0.4
Accuracy: 0.999640
Precision: 0.950495
Recall: 0.727273
f1: 0.824034
auc_score: 0.863614
